# Tasks Test 

In [386]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.linear_model import LinearRegression
from src import *

In [387]:
data_root = '/Users/huayuzhu/Desktop/exam/raw_data/daily'
output_dir = '/Users/huayuzhu/Desktop/exam/'

In [398]:
def get_data_from_csv(file_path):
    try:
        data = pd.read_csv(file_path, index_col=0).T
        data.index = pd.to_datetime(data.index)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return data

In [388]:
S_DQ_RET = get_data_from_csv(f'{data_root}/S_DQ_RET.csv')
S_905_DQ_RET =  get_data_from_csv(f'{data_root}/905S_DQ_RET.csv')
S_DQ_MV = get_data_from_csv(f'{data_root}/S_DQ_MV.csv')
S_RESTRICT = get_data_from_csv(f'{data_root}/S_RESTRICT.csv')
F7_26 =  get_data_from_csv(f'{data_root}/F7_26.csv')
F7_27 =  get_data_from_csv(f'{data_root}/F7_27.csv')
S_DQ_OPEN = get_data_from_csv(f'{data_root}/S_DQ_OPEN.csv')

## Task1 

### 1.1 Signle Factor Backtest 

In [391]:
test1 = TestInfo()
factor27 = OneFactorTest(F7_27)
factor27.plot_comparison_with_benchmark(test1,'F7_27')
factor27.compare_with_benchmark(test1,'F7_27',False).head(5)

factor26 = OneFactorTest(F7_26)
factor26.plot_comparison_with_benchmark(test1,'F7_26')
factor26.compare_with_benchmark(test1,'F7_26',False).head(5)

TypeError: OneFactorTest.plot_comparison_with_benchmark() takes 2 positional arguments but 3 were given

In [383]:
def mad(factor):
    me = np.median(factor)
    mad = np.median(abs(factor-me))
    up = me + (3*1.4826*mad)
    down = me - (3*1.4826*mad)
    factor = np.where(factor>up,up,factor)
    factor = np.where(factor<down,down,factor)
    return factor

def stand(factor):
    mean = factor.mean()
    std = factor.std()
    return (factor-mean)/std

def neutralize_factors(df_factor, df_MV, name, output = True):
    df_f1 = df_factor.copy()
    for col in df_factor.columns:
        factor_col = df_f1[col]
        if factor_col.isnull().all():
            continue
        df_f1[col] = mad(factor_col)
        df_f1[col] = stand(df_f1[col])
        df_f1[col].replace([np.inf, -np.inf], np.nan, inplace=True) 
        
        x = df_MV[col].dropna()
        y = df_f1[col].loc[x.index]   
        valid_idx = y.notna()
        x = x[valid_idx].to_frame()
        y = y[valid_idx]

 
        if not x.empty and not y.isin([np.inf, -np.inf, np.nan]).any():
            L1 = LinearRegression()
            L1.fit(x, y)
            predictions = pd.Series(L1.predict(df_MV[col].dropna().to_frame()), index=df_MV[col].dropna().index)
            df_f1[col].update(df_f1[col] - predictions.reindex(df_f1.index))

    if output_dir:
        df_f1.to_csv(f'{output_dir}/{name}_neutralize.csv')

    return df_f1

In [384]:
neutralize_factors(F7_27,S_DQ_MV,'F7_27',output = False)

,1,2,3,4,5,6,7,8,9,10,...,873593,873665,873679,873690,873693,873703,873726,873749,873806,873833
2015-01-05,-0.186287,-0.200464,NaN,NaN,NaN,-0.070613,NaN,-3.696805,-0.489520,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,-0.179108,-0.680094,NaN,NaN,NaN,-0.128504,NaN,-3.698805,-0.489520,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,-0.170135,-0.632346,NaN,NaN,NaN,-0.125987,NaN,-3.706497,-0.483469,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,-0.154581,-0.607480,NaN,NaN,NaN,-0.146123,NaN,-3.717420,-0.483150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-09,-0.158171,-0.602023,NaN,NaN,NaN,-0.166259,NaN,-3.719421,-0.489042,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,1.172304,0.825364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-27,1.152242,0.819880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-28,1.150472,0.817139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-29,1.156963,0.833589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [385]:
F7_27

,1,2,3,4,5,6,7,8,9,10,...,873593,873665,873679,873690,873693,873703,873726,873749,873806,873833
2015-01-05,-0.054025,-0.012440,NaN,NaN,NaN,0.0,NaN,-0.813953,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,-0.054025,-0.021230,NaN,NaN,NaN,0.0,NaN,-0.813953,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,-0.054025,-0.020482,NaN,NaN,NaN,0.0,NaN,-0.813953,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,-0.054025,-0.020483,NaN,NaN,NaN,0.0,NaN,-0.813953,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-09,-0.054025,-0.020483,NaN,NaN,NaN,0.0,NaN,-0.813953,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,0.242848,0.007981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-27,0.242848,0.007981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-28,0.242848,0.007981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-29,0.242848,0.007981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
